### This project performed text classification of medical records by using fine-tuning LLMs.

In [1]:
import os
import pandas as pd
import transformers
from datasets import load_dataset
from transformers import pipeline

C:\Users\Sealion\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [60]:
# conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

In [3]:
# import torch
# torch.cuda.get_device_name(0)

In [4]:
# DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
med_dataset = load_dataset("rungalileo/medical_transcription_4", cache_dir="../datasets/")  
med_dataset  

Found cached dataset parquet (C:/Users/Sealion/Desktop/SOS_JOB/datasets/rungalileo___parquet/rungalileo--medical_transcription_4-cc509920750fa75b/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 4499
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 500
    })
})

In [6]:
med_total = pd.concat([pd.DataFrame.from_dict(med_dataset['train']), pd.DataFrame.from_dict(med_dataset['test'])], axis = 0)
display(med_total.head(5)), med_total.shape

,id,text,label
0,0,"2-D STUDY,1. Mild aortic stenosis, widely calc...",0
1,1,"PREOPERATIVE DIAGNOSES: , Dysphagia and esopha...",3
2,2,"CHIEF COMPLAINT:, The patient comes for three...",0
3,3,"PROCEDURE: , Bilateral L5, S1, S2, and S3 radi...",3
4,4,"DISCHARGE DIAGNOSES:,1. Chronic obstructive pu...",0


(None, (4999, 3))

In [7]:
med_total['label'].value_counts()

3    1640
0    1228
2    1149
1     982
Name: label, dtype: int64

In [8]:
med_train = med_total[med_total['label'] == 0][['text', 'label']].reset_index(drop=True)
med_train.head(5)

,text,label
0,"2-D STUDY,1. Mild aortic stenosis, widely calc...",0
1,"CHIEF COMPLAINT:, The patient comes for three...",0
2,"DISCHARGE DIAGNOSES:,1. Chronic obstructive pu...",0
3,"SUBJECTIVE:, The patient is a 2-year-old litt...",0
4,"REASON FOR EVALUATION: , The patient is a 37-y...",0


In [9]:
set(med_train['label'])

{0}

In [10]:
# med_train['text'][1]

In [11]:
med_test = med_total[med_total['label'] == 2][['text', 'label']].reset_index(drop=True)
med_test.head(5)

,text,label
0,"INDICATION:, Coronary artery disease, severe ...",2
1,The patient was told that the injection may ca...,2
2,"HISTORY OF PRESENT ILLNESS:, The patient is a ...",2
3,"INDICATION: , Chest pain.,TYPE OF TEST: , Aden...",2
4,"HISTORY AND CLINICAL DATA: ,The patient is an...",2


In [64]:
med_test['text'][10]

'CHIEF COMPLAINT:,  Arm and leg jerking.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 10-day-old Caucasian female here for approximately 1 minute bilateral arm and leg jerks, which started at day of life 1 and have occurred 6 total times since then.  Mom denies any apnea, perioral cyanosis, or color changes.  These movements are without any back arching.  They mainly occur during sleep, so mom is unaware of any eye rolling.  Mom is able to wake the patient up during this periods and stop the patient\'s extremity movements.,Otherwise, this patient has been active, breast-feeding well, although she falls asleep at the breast.  She is currently taking in 15 to 20 minutes of breast milk every 2 to 3 hours.  She is having increased diapers up to 8 wet and 6 to 7 dirty-yellow stools per day.,REVIEW OF SYSTEMS:,  Negative fever, negative fussiness, tracks with her eyes, some sneezing and hiccups.  This patient has developed some upper airway congestion in the past day.  She has not had any 

In [13]:
set(med_test['label'])

{2}

In [14]:
med_unsuper = med_total[med_total['label'] == 2][['text', 'label']].reset_index(drop=True)
med_unsuper.head(5)

,text,label
0,"INDICATION:, Coronary artery disease, severe ...",2
1,The patient was told that the injection may ca...,2
2,"HISTORY OF PRESENT ILLNESS:, The patient is a ...",2
3,"INDICATION: , Chest pain.,TYPE OF TEST: , Aden...",2
4,"HISTORY AND CLINICAL DATA: ,The patient is an...",2


In [15]:
med_unknown = med_total[med_total['label'] == 1][['text', 'label']].reset_index(drop=True)
med_unknown.head(5)

,text,label
0,"SUBJECTIVE:, The patient is brought in by an ...",1
1,"PREOPERATIVE DIAGNOSES:,1. Left carpal tunnel...",1
2,"As you know, the patient is a 50-year-old righ...",1
3,"HISTORY: , The patient is a 56-year-old right-...",1
4,"ADMISSION DIAGNOSIS: , Left hip fracture.,CHIE...",1


In [16]:
med_unknown['text'][0]

"SUBJECTIVE:,  The patient is brought in by an assistant with some of his food diary sheets.  They wonder if the patient needs to lose anymore weight.,OBJECTIVE:,  The patient's weight today is 186-1/2 pounds, which is down 1-1/2 pounds in the past month.  He has lost a total of 34-1/2 pounds.  I praised this.  I went over his food diary and praised all of his positive food choices reported, especially his use of sugar-free Kool-Aid, sugar-free pudding, and diet pop.  I encouraged him to continue all of that, as well as his regular physical activity.,ASSESSMENT:,  The patient is losing weight at an acceptable rate.  He needs to continue keeping a food diary and his regular physical activity.,PLAN:,  The patient plans to see Dr. XYZ at the end of May 2005.  I recommended that they ask Dr. XYZ what weight he would like for the patient to be at.  Follow up will be with me June 13, 2005.diets and nutritions, weight, kool-aid, food diary sheets, diary sheets, physical activity, food diary, 

In [44]:
# med_unknown['text'][2]

In [18]:
med_train.shape, med_test.shape, med_unsuper.shape, med_unknown.shape

((1228, 2), (1149, 2), (1149, 2), (982, 2))

In [19]:
import datasets
from datasets import Dataset, DatasetDict
newmed_dataset = DatasetDict({
    "train": Dataset.from_pandas(med_train),
    "test": Dataset.from_pandas(med_test),
    "unsupervised":Dataset.from_pandas(med_unsuper),
#     "unknown": Dataset.from_pandas(med_unknown)
    })
newmed_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1228
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1149
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 1149
    })
})

In [23]:
model_checkpoint = "t5-small"

In [24]:
# load the tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_checkpoint, cache_dir="../datasets/"
)  

In [25]:
def to_tokens(
    tokenizer: transformers.models.t5.tokenization_t5_fast.T5TokenizerFast, label_map: dict
) -> callable:
    def apply(x) -> transformers.tokenization_utils_base.BatchEncoding:
        """From a formatted dataset `x` a batch encoding `token_res` is created."""
        target_labels = [label_map[y] for y in x["label"]]
        token_res = tokenizer(
            x["text"],
            text_target=target_labels,
            return_tensors="np",
            truncation=True,
            padding=True,
        )
        return token_res

    return apply

In [ ]:
med_label_lookup = {0: "Medical Records", 2:"Internal Medicine", 3: "Surgery"}

In [26]:
med_to_tokens = to_tokens(tokenizer, med_label_lookup)
tokenized_dataset = newmed_dataset.map(
    med_to_tokens, batched=True, remove_columns=["text", "label"]
)

Map:   0%|          | 0/1228 [00:00<?, ? examples/s]

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [27]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1228
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1149
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1149
    })
})

In [29]:
import tempfile
tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name

In [30]:
checkpoint_name = "test-trainer"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
training_args = transformers.TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,  
    per_device_train_batch_size=16,
    optim="adamw_torch",
    report_to=["tensorboard"],
)

In [31]:
# load the pre-trained model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, cache_dir="../datasets/"
)  

In [32]:
# Batch
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
trainer = transformers.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [33]:
tensorboard_display_dir = "../datasets/tensorboard"

In [34]:
# remove AppData/Local/Temp/.tensorboard-info
%load_ext tensorboard
%tensorboard --logdir '{tensorboard_display_dir}'

Reusing TensorBoard on port 6006 (pid 9236), started 0:09:13 ago. (Use '!kill 9236' to kill it.)

In [35]:
# save model to the local checkpoint
trainer.train()
trainer.save_model()
trainer.save_state()

***** Running training *****
  Num examples = 1228
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 77
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer
Configuration saved in C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\config.json
Configuration saved in C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\generation_config.json
Model weights saved in C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\pytorch_model.bin
tokenizer config file saved in C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\tokenizer_config.json
Special tokens file saved in C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\special_tokens_map.json
Copy vocab file to C:\Users\Sealion\AppData\Local\Temp\tmprh8iqfd7\test-trainer\spiece.model


In [36]:
# persist the fine-tuned model to DBFS
final_model_path = f"../datasets/med_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

Saving model checkpoint to ../datasets/med_fine_tuning/test-trainer
Configuration saved in ../datasets/med_fine_tuning/test-trainer\config.json
Configuration saved in ../datasets/med_fine_tuning/test-trainer\generation_config.json
Model weights saved in ../datasets/med_fine_tuning/test-trainer\pytorch_model.bin
tokenizer config file saved in ../datasets/med_fine_tuning/test-trainer\tokenizer_config.json
Special tokens file saved in ../datasets/med_fine_tuning/test-trainer\special_tokens_map.json
Copy vocab file to ../datasets/med_fine_tuning/test-trainer\spiece.model


In [37]:
fine_tuned_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)

loading configuration file ../datasets/med_fine_tuning/test-trainer\config.json
Model config T5Config {
  "_name_or_path": "../datasets/med_fine_tuning/test-trainer",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
 

In [51]:
documents = ["""
The patient is brought in by an assistant with some of his food diary sheets.  They wonder if the patient needs to lose anymore weight.,OBJECTIVE:,  The patient's weight today is 186-1/2 pounds, which is down 1-1/2 pounds in the past month.  He has lost a total of 34-1/2 pounds.  I praised this.  I went over his food diary and praised all of his positive food choices reported, especially his use of sugar-free Kool-Aid, sugar-free pudding, and diet pop.  I encouraged him to continue all of that, as well as his regular physical activity.,ASSESSMENT:,  The patient is losing weight at an acceptable rate.  He needs to continue keeping a food diary and his regular physical activity.,PLAN:,  The patient plans to see Dr. XYZ at the end of May 2005.  I recommended that they ask Dr. XYZ what weight he would like for the patient to be at.  Follow up will be with me June 13, 2005.diets and nutritions, weight, kool-aid, food diary sheets, diary sheets, physical activity, food diary, dietary, sheets, diary, food
"""]
inputs = tokenizer(documents, return_tensors="pt", truncation=True, padding=True)
pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

C:\Users\Sealion\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [52]:
pdf = pd.DataFrame(
    zip(documents, tokenizer.batch_decode(pred, skip_special_tokens=True)), 
    columns=["documents", "class"],
)
display(pdf)

,documents,class
0,\nThe patient is brought in by an assistant wi...,Medical Records


In [53]:
pdf['class']

0    Medical Records
Name: class, dtype: object

[Note]: Many thanks to Databricks' (edx courses) method and idea.